In [42]:
import numpy as np
import matplotlib.pyplot as plt
from Datagen import sim1d
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline


In [43]:
L1 = sim1d(rho_l=2460.0, rho_s=2710.0, k_l=104.0, k_s= 96.2, cp_l=1245.3, cp_s=963.0, \
            t_surr=298.0, L_fusion=389000, temp_init=913.0, htc_l=10.0,htc_r= 12.0, length =15.0e-3)
                
L2 = sim1d(rho_l=2460.0, rho_s=2710.0, k_l=104.0, k_s= 96.2, cp_l=1245.3, cp_s=963.0, \
            t_surr=298.0, L_fusion=389000, temp_init=913.0, htc_l=12.0,htc_r= 12.0, length =15.0e-3)
L3 = sim1d(rho_l=2460.0, rho_s=2710.0, k_l=104.0, k_s= 96.2, cp_l=1245.3, cp_s=963.0, 
               t_surr=298.0, L_fusion=389000, temp_init=913.0, htc_l=15.0,htc_r= 12.0, length =15.0e-3)          


def strat_sampler (x,t,percent):

    l1 = len(x)
    indices_1 = list(range(l1))

    l2 = len(t)
    indices_2 = list(range(l2))

    bins_l1 = l1 //3
    bins_l2 = l2 //3

    l1_indices_1 = indices_1[:bins_l1] 
    l1_indices_2 = indices_1[bins_l1:2*bins_l1]
    l1_indices_3 = indices_1[2*bins_l1:]

    l2_indices_1 = indices_2[:bins_l2]
    l2_indices_2 = indices_2[bins_l2:2*bins_l2]
    l2_indices_3 = indices_2[2*bins_l2:]

    bin1 = np.random.choice(l1_indices_1, int(percent*bins_l1), replace=False)
    bin2 = np.random.choice(l1_indices_2, int(percent*bins_l1), replace=False)
    bin3 = np.random.choice(l1_indices_3, int(percent*bins_l1), replace=False)


    bin4 = np.random.choice(l2_indices_1, int(percent*bins_l2), replace=False)
    bin5 = np.random.choice(l2_indices_2, int(percent*bins_l2), replace=False)
    bin6 = np.random.choice(l2_indices_3, int(percent*bins_l2), replace=False)

    index1 = np.concatenate((bin1,bin2,bin3))
    index2 = np.concatenate((bin4,bin5,bin6))

    return x[index1] , t[index2]


htc_l_1 = 10.0
htc_l_2 = 12.0
htc_l_3 = 15.0

T1,space_1,time_1 = L1[0],L1[1],L1[2]
T2,space_2,time_2 = L2[0],L2[1],L2[2]
T3,space_3,time_3 = L3[0],L3[1],L3[2]
    


space_1, time_1 = np.meshgrid(space_1, time_1)
space_2, time_2 = np.meshgrid(space_2, time_2)
space_3, time_3 = np.meshgrid(space_3, time_3)

space_1 = space_1.flatten()
time_1 = time_1.flatten()
htc_l_1 = np.ones(space_1.shape)*htc_l_1

space_2 = space_2.flatten()
time_2 = time_2.flatten()
htc_l_2 = np.ones(space_2.shape)*htc_l_2

space_3 = space_3.flatten()
time_3 = time_3.flatten()
htc_l_3 = np.ones(space_3.shape)*htc_l_3

Temp1 = T1.flatten()
Temp2 = T2.flatten()
Temp3 = T3.flatten()

space_all = np.concatenate((space_1,space_2,space_3),axis=0)
time_all = np.concatenate((time_1,time_2,time_3),axis=0)
Temp_all = np.concatenate((Temp1,Temp2,Temp3),axis=0)
htc_l_all = np.concatenate((htc_l_1,htc_l_2,htc_l_3),axis=0)


print(space_all.shape,time_all.shape,Temp_all.shape,htc_l_all.shape)


inputs = np.column_stack((space_all, time_all,htc_l_all))
outputs = Temp_all

# print(inputs.shape)
# print(outputs.shape)

Stable
Stable
Stable
(4720500,) (4720500,) (4720500,) (4720500,)


In [47]:
percent = 5/100
inputs_sampled, outputs_sampled = strat_sampler(inputs,outputs,percent)
print(inputs_sampled.shape,outputs_sampled.shape)

(236025, 3) (236025,)


In [45]:
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs_sampled, outputs_sampled, test_size=0.2, random_state=42)


## train polynomial regression

In [46]:
degree = 3
scaler = StandardScaler()
poly = PolynomialFeatures(degree,include_bias=False)

model = make_pipeline(scaler,poly, LinearRegression())    

model.fit(inputs_train, outputs_train)

print(model.score(inputs_test, outputs_test))




0.019607991313964623
